In [1]:
from pgvector.sqlalchemy import VECTOR
from sqlalchemy import select
from sqlalchemy import create_engine
import pandas as pd

In [2]:
import getpass
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker, declarative_base

# --- 1. Connection Parameters ---
DB_USER = "postgres.wfsrsffyzpagawyjkslw"
DB_PASS = getpass.getpass("Enter Supabase Password: ") # Secure input
DB_HOST = "aws-1-us-east-2.pooler.supabase.com"
DB_PORT = "5432"
DB_NAME = "postgres"

# Build the connection string (ensure it starts with postgresql://)
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# --- 2. Create the Engine ---
# 'echo=True' will print the SQL being generated, great for learning!
engine = create_engine(DATABASE_URL, echo=False)

# --- 3. Create a Session Factory ---
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# --- 4. Define the Base for Models ---
Base = declarative_base()

# --- 5. Test the Connection ---
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT greeting FROM (SELECT 'Connection Successful!' AS greeting) AS sub;"))
        print(result.all()[0][0])
except Exception as e:
    print(f"Error connecting: {e}")


Enter Supabase Password:  ········


Connection Successful!


In [9]:
from sqlalchemy import Column, Integer, Text, create_engine
from sqlalchemy.orm import declarative_base, sessionmaker
from pgvector.sqlalchemy import Vector

Base = declarative_base()

class MontessoriChunk(Base):
    __tablename__ = 'montessori_data'
    id = Column(Integer, primary_key=True)
    content = Column(Text)         # The actual paragraph
    embedding = Column(Vector(384)) # The mathematical representation

#Base.metadata.create_all(engine)

print("Table 'montessori_data' created successfully!")


Table 'montessori_data' created successfully!


In [9]:
from sqlalchemy import inspect

inspector = inspect(engine)
if "montessori_data" in inspector.get_table_names():
    print("Verification: Table is present in the database.")
else:
    print("Verification: Table not found.")

Verification: Table is present in the database.


In [6]:
from sentence_transformers import SentenceTransformer

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
model = SentenceTransformer('all-MiniLM-L6-v2')
paragraphs = [
    "The greatest sign of success for a teacher is to be able to say, 'The children are now working as if I did not exist.'",
    "Free choice is one of the highest of all the mental processes.",
    "Education is a natural process carried out by the human individual, and is acquired not by listening to words, but by experiences in the environment.",
    "The child who has never learned to act alone, to direct his own actions, to govern his own will, is anticipated by others in all his movements."
]


In [10]:
Session = sessionmaker(bind=engine)
with Session() as session:
    for p in paragraphs:
        # Turn text into a vector
        vector = model.encode(p).tolist() 
        
        # Create an instance of your MontessoriChunk class
        chunk = MontessoriChunk(content=p, embedding=vector)
        session.add(chunk)
    
    session.commit()
    print(f"Successfully embedded and saved {len(paragraphs)} paragraphs.")

Successfully embedded and saved 4 paragraphs.


In [11]:
# 1. Define your search query
search_query = "sign of success" # Replace with your desired search query

# 2. Convert the query into a vector using the SAME model
query_vector = model.encode(search_query).tolist()

# 3. Search the database
with Session() as session:
    # Use the cosine_distance operator (<=>) to find the "nearest" matches
    # Distance of 0 means identical; higher numbers mean more different
    results = session.query(MontessoriChunk).order_by(
        MontessoriChunk.embedding.cosine_distance(query_vector)
    ).limit(2).all()

    print(f"Search Results for: '{search_query}'\n" + "-"*50)
    for row in results:
        print(f"Match: {row.content}\n")


Search Results for: 'sign of success'
--------------------------------------------------
Match: The greatest sign of success for a teacher is to be able to say, 'The children are now working as if I did not exist.'

Match: Free choice is one of the highest of all the mental processes.



In [10]:
# A collection of distinct Montessori themes
texts = [
    """The teacher's task is not to talk, but to prepare and arrange a series of motives for cultural activity in a special environment made for the child. The teacher should be like a custodian who protects the environment, keeping it orderly and beautiful, so the child can work undisturbed.""",
    
    """Our aim is not only to make the child understand, and still less to force him to memorize, but so to touch his imagination as to enthuse him to his innermost core. We do not want compliant students; we want sparked souls who seek knowledge for themselves.""",
    
    """Discipline is not something that is granted or imposed from the outside. It is a slow, internal construction that the child builds through work. A child is disciplined when he can guide his own actions and control his own impulses, not when he is silent because of a command.""",
    
    """The hand is the instrument of intelligence. The child needs to manipulate objects and to gain experience by using his hands. It is through the movement of the hands that the mind is formed and the personality develops its distinct characteristics.""",
    
    """The environment itself must be 'proportional' to the child. The furniture must be light so that he can move it, the washstands low so he can reach them, the brushes small enough for his hands. In a world made for adults, the child is a foreigner; in a prepared environment, he is a master.""",
    
    """Observation is the fundamental tool of the educator. We must stand back and watch the child's spontaneous development without the desire to interfere or direct. Only through scientific observation can we discover what the child truly needs to progress.""",
    
    """Adults often treat children as if they were empty vessels to be filled. But the child is not a passive recipient; he is the 'creator of the man.' Every child has an internal blueprint for his own development, and our role is merely to remove the obstacles in his path.""",
    
    """When a child concentrates on a task—whether it is pouring water or polishing a shoe—he enters a state of 'polarization of attention.' This concentration is the beginning of his spiritual life. We must never interrupt a child who is concentrating, for this is the moment of greatest growth.""",
    
    """Reward and punishment are the instruments of the slave-driver. They lead to an artificial development of the child's personality. True interest comes from the work itself, and the satisfaction of completing a task is the only reward a free child requires.""",
    
    """The child has a different relationship with time than the adult. An adult wants to finish a task quickly to reach a goal; a child works for the sake of the process. For the child, the work is the goal, and time is irrelevant to his internal satisfaction."""
]

model = SentenceTransformer('all-MiniLM-L6-v2')
# Simple Chunking & Insertion
from sqlalchemy.orm import Session

with Session(engine) as session:
    for text_body in texts:
        # Generate embedding
        vector = model.encode(text_body).tolist()
        # Add to DB
        session.add(MontessoriChunk(content=text_body, embedding=vector))
    
    session.commit()
    print(f"Added {len(texts)} new complex passages to the database.")


Added 10 new complex passages to the database.


In [6]:
from sentence_transformers import SentenceTransformer

In [7]:
print('done')

done


In [11]:
def test_search(query, limit=3):
    query_vector = model.encode(query).tolist()
    with Session(engine) as session:
        results = session.query(MontessoriChunk).order_by(
            MontessoriChunk.embedding.cosine_distance(query_vector)
        ).limit(limit).all()
        
        print(f"\nQUERY: '{query}'")
        print("-" * 60)
        for i, res in enumerate(results):
            print(f"[{i+1}] {res.content[:250]}...")

In [ ]:

test_search("learning by doing and physical touch") 
# (Should find the 'hand is the instrument' passage)

test_search("the teacher's role in the classroom") 
# (Should find 'custodian' and 'observation' passages)

test_search("motivation without prizes or threats") 
# (Should find 'reward and punishment' and 'satisfaction' passages)

test_search("creating a space specifically for kids") 
# (Should find the 'proportional furniture' passage)

In [12]:

test_search("learning by doing and physical touch") 


QUERY: 'learning by doing and physical touch'
------------------------------------------------------------
[1] The hand is the instrument of intelligence. The child needs to manipulate objects and to gain experience by using his hands. It is through the movement of the hands that the mind is formed and the personality develops its distinct characteristics....
[2] Education is a natural process carried out by the human individual, and is acquired not by listening to words, but by experiences in the environment....
[3] Our aim is not only to make the child understand, and still less to force him to memorize, but so to touch his imagination as to enthuse him to his innermost core. We do not want compliant students; we want sparked souls who seek knowledge for themse...


In [13]:

test_search("the teacher's role in the classroom") 
# (Should find 'custodian' and 'observation' passages)



QUERY: 'the teacher's role in the classroom'
------------------------------------------------------------
[1] The teacher's task is not to talk, but to prepare and arrange a series of motives for cultural activity in a special environment made for the child. The teacher should be like a custodian who protects the environment, keeping it orderly and beautiful...
[2] The greatest sign of success for a teacher is to be able to say, 'The children are now working as if I did not exist.'...
[3] Observation is the fundamental tool of the educator. We must stand back and watch the child's spontaneous development without the desire to interfere or direct. Only through scientific observation can we discover what the child truly needs to progres...


In [14]:

test_search("what is the teacher's role in the classroom") 
# (Should find 'custodian' and 'observation' passages)



QUERY: 'what is the teacher's role in the classroom'
------------------------------------------------------------
[1] The teacher's task is not to talk, but to prepare and arrange a series of motives for cultural activity in a special environment made for the child. The teacher should be like a custodian who protects the environment, keeping it orderly and beautiful...
[2] The greatest sign of success for a teacher is to be able to say, 'The children are now working as if I did not exist.'...
[3] Observation is the fundamental tool of the educator. We must stand back and watch the child's spontaneous development without the desire to interfere or direct. Only through scientific observation can we discover what the child truly needs to progres...
